In [326]:
library(igraph)

In [327]:
#build personal networks 
createnet <- function(node_id ){
        filename <- paste('./gplus/',node_id,'.edges',sep = "")
        print(filename)
        g <- read.graph(filename,format = 'ncol', directed=TRUE)
        g <- add_vertices(g, nv=1, name = node_id)
        new_edges = c()
        for (id in 1:(vcount(g)-1)){
            new_edges <- c(new_edges, c(vcount(g), id))
        }
        
        #print(vcount(g))
        g <- add_edges(g, new_edges)
        return (g)
}




In [328]:
g1 <- createnet(node_id = '109327480479767108490')
g2 <- createnet(node_id = '115625564993990145546')
g3 <- createnet(node_id = '101373961279443806744')

[1] "./gplus/109327480479767108490.edges"
[1] "./gplus/115625564993990145546.edges"
[1] "./gplus/101373961279443806744.edges"


In [329]:
# return c(unique number N in circles, length of circle file, vertex in circle file)
countN <- function(node_id){
    filename <- paste('./gplus/',node_id,'.circles',sep = "")
    circles <- file(filename, 'r')
    circle_lines <- readLines(circles)
    c_vertex = c()
    for (i in 1:length(circle_lines)){
        match <- gregexpr('[0-9]{21}', circle_lines[i])
        matches <- regmatches(circle_lines[i], match)
        #print(length((matches)[[1]]))
        for (j in 1:length((matches)[[1]])){
            c_vertex <- c(c_vertex, matches[[1]][j])
        }
        #print(length(c_vertex))
    }
    #print(length(unique(c_vertex)))
    return (c((length(unique(c_vertex))), length(c_vertex), c_vertex))
}

N1 <- countN('109327480479767108490')[1]
N2  <- countN('115625564993990145546')[1]
N3  <- countN('101373961279443806744')[1]

L1 <- countN('109327480479767108490')[2]
v1 <- countN('109327480479767108490')[3:L1 + 2]

L2 <- countN('115625564993990145546')[2]
v2 <- countN('115625564993990145546')[3:L2 + 2]

L3 <- countN('101373961279443806744')[2]
v3 <- countN('101373961279443806744')[3:L3 + 2]


In [330]:
#calculate H(C)
hc <- function(node_id, N){
    filename <- paste('./gplus/',node_id,'.circles',sep = "")
    circles <- file(filename, 'r')
    circle_lines <- readLines(circles)
    sum = 0;
    for(i in 1:length(circle_lines)){
        match <- gregexpr('[0-9]{21}', circle_lines[i])
        matches <- regmatches(circle_lines[i], match)
        ai = length((matches)[[1]])
        #print(ai)
        N <- as.numeric(N)
        sum = sum - ai/N*log10(ai/N)
    }
    return (sum)
}

    

In [331]:
hc1 <- hc('109327480479767108490',N1)
hc2 <- hc('115625564993990145546',N2)
hc3 <- hc('101373961279443806744',N3)

In [359]:
print(hc1)
print(hc2)
print(hc3)

[1] 0.4563477
[1] 3.676366
[1] 0.166908


In [355]:
community1 <- cluster_walktrap(g1)
community2 <- cluster_walktrap(g2)
community3 <- cluster_walktrap(g3)

#store community members into vectors
community_vector <- function(community){
    #store community memmber into a vector
    m = c()
    for(i in 1:length((membership(community)))){ m = c(m, (membership(community))[i])}
    #calculate number of cluster
    L_comm = length(m)
    c_cluster = c()
    for(i in 1:L_comm){
        c_cluster = c(c_cluster, m[[i]])
        cluster_number = length(unique(c_cluster))
    }
    #print(cluster_number)
    #store vertexs into each cluster
    mylist <- list()
    for(k in 1:cluster_number){
        mylist[[paste('vector_', k, sep = "")]] <- c() 
    }
    #print(mylist)
    
    v = c()
    for(j in 1:L_comm){
        match <- gregexpr('[0-9]{21}', m[j])
        matches <- regmatches(m[j], match)
        for(k in 1:cluster_number){
            if (m[[j]] == k){               
                mylist[[paste('vector_', k, sep = "")]] <- c(mylist[[paste('vector_', k, sep = "")]],matches) 
                                                             
            }
        }
        #print(matches)
        
    }
    #print(mylist)
    return (mylist)
}


cv1 <- community_vector(community1)
cv2 <- community_vector(community2)
cv3 <- community_vector(community3)



In [356]:
#calculate H(K)
hk <- function(community_list, N, circle_ver){
    #print(circle_lines[3])
    sum = 0
    #print(length(community_list))
    for(i in 1:length(community_list)){
        com_vec = names(community_list[[paste('vector_', i, sep = "")]])
        community_vec = c()
        for( n in com_vec){ community_vec <- c(community_vec, n)}        
        inter <- intersect(community_vec,circle_ver)
        bi <- length(inter)
        #print(bi)
        #bi <-  as.numeric(bi)
        N <- as.numeric(N)
        if(bi==0){ 
            sum = sum
        }else{sum = sum - bi/N*log10(bi/N)}       
    }
    return (sum)    
}

hk1 <- hk(cv1,N1, v1 )
hk2 <- hk(cv2,N2, v2 )
hk3 <- hk(cv3,N3, v3 )

In [358]:
print(hk1)
print(hk2)
print(hk3)

[1] 0.4365564
[1] 0.4695553
[1] 0.2142508


In [383]:
hck <- function(N,cv, v, node_id){
    filename <- paste('./gplus/',node_id,'.circles',sep = "")
    circles <- file(filename, 'r')
    circle_lines <- readLines(circles)
    sum = 0
    for(j in 1:length(cv)){
        #for each community
        com_vec = names(cv[[paste('vector_', j, sep = "")]])
        community_vec = c()
        for( n in com_vec){ community_vec <- c(community_vec, n)} 
        N <- as.numeric(N)
        #calculate bj
        inter <- intersect(community_vec,v)
        bj <- length(inter)
        
        
        #for each circle
        for(i in 1:length(circle_lines)){
            #output vertex vector in each circle
            c_vertex <- c()
            match <- gregexpr('[0-9]{21}', circle_lines[i])
            matches <- regmatches(circle_lines[i], match)
            for (k in 1:length((matches)[[1]])){
                 c_vertex <- c(c_vertex, matches[[1]][k])
            }
            #intersection of community j and circle i
            interji <- intersect(community_vec,c_vertex)
            cji <- length(interji)
            if(cji == 0){
                sum = sum
            }else{
                sum = sum - cji/N*log10(cji/bj)}
                 
        }
        
    }
    return(sum)
}

hck1 <- hck(N1,cv1,v1,'109327480479767108490' )
hck2 <- hck(N2,cv2,v2,'115625564993990145546' )
hck3 <- hck(N3,cv3,v3,'101373961279443806744' )

In [384]:
print(hck1)
print(hck2)
print(hck3)

[1] 0.06759188
[1] 2.015052
[1] 0.1662627


In [386]:
hkc <- function(N,cv, v, node_id){
    filename <- paste('./gplus/',node_id,'.circles',sep = "")
    circles <- file(filename, 'r')
    circle_lines <- readLines(circles)
    sum = 0
    for(i in 1:length(circle_lines)){
       c_vertex <- c()
       match <- gregexpr('[0-9]{21}', circle_lines[i])
       matches <- regmatches(circle_lines[i], match)
       ai = length((matches)[[1]])
       for (k in 1:length((matches)[[1]])){
            c_vertex <- c(c_vertex, matches[[1]][k])
       } 
       for(j in 1:length(cv)){
        #for each community
        com_vec = names(cv[[paste('vector_', j, sep = "")]])
        community_vec = c()
        for( n in com_vec){ community_vec <- c(community_vec, n)} 
        N <- as.numeric(N)
        #intersection of community j and circle i
        interji <- intersect(community_vec,c_vertex)
        cji <- length(interji)
        if(cji == 0){
                sum = sum
        }else{
                sum = sum - cji/N*log10(cji/ai)}
        
        
       }
    
    }
    return(sum)
}

hkc1 <- hkc(N1,cv1,v1,'109327480479767108490' )
hkc2 <- hkc(N2,cv2,v2,'115625564993990145546' )
hkc3 <- hkc(N3,cv3,v3,'101373961279443806744' )

In [387]:
print(hkc1)
print(hkc2)
print(hkc3)

[1] 0.2925478
[1] 2.077295
[1] 0.536535


In [391]:
h1 <- (1 - hck1/hc1)
h2 <- (1 - hck2/hc2)
h3 <- (1 - hck3/hc3)
c1 <- (1 - hkc1/hk1)
c2 <- (1 - hkc2/hk2)
c3 <- (1 - hkc3/hk3)


In [393]:
print(h1)
print(c1)

print(h2)
print(c2)

print(h3)
print(c3)



[1] 0.8518851
[1] 0.3298739
[1] 0.4518903
[1] -3.423962
[1] 0.003866707
[1] -1.504238
